# IR PROJECT CODE

Members:

*   Omer
*   Lucas
*   Muhamad

# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

LLM MODEL GEMMA3 270M setup for question answering

Based on: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=pCqnaKmlO1U9

In [1]:
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install pyterrier
!pip install pyterrier[java]
!pip install gensim
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install nltk
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from unsloth import FastLanguageModel

import torch
import torch.nn.functional as F
import pyterrier as pt
from pyterrier.measures import P, MAP, R, nDCG
from pyterrier_t5 import MonoT5ReRanker
import os
import json
import pandas as pd
import shutil
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import numpy as np
import pickle
from gensim.models import KeyedVectors

import csv
import xml.etree.ElementTree as ET
from xml.dom import minidom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2026.1.4 requ

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o validation.json

import json

files = ["train.json", "validation.json", "test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1073  100  1073    0     0   4040      0 --:--:-- --:--:-- --:--:--  4049
100 71.5M  100 71.5M    0     0  33.2M      0  0:00:02  0:00:02 --:--:-- 87.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1079  100  1079    0     0   3612      0 --:--:-- --:--:-- --:--:--  3620
100 1315M  100 1315M    0     0  61.9M      0  0:00:21  0:00:21 --:--:-- 69.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1073  100  1073    0     0   3511      0 --:--:-- --:--:-- --:--:--  3506
100 71.8M  100 71.8M    0     0  34.5M      0  0:00:02  0:00:02 --:--:-- 79.2M

===== train.json =====
Preview of first 500 charact

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [3]:
import json
import os

inputs = ["train.json", "validation.json", "test.json"]
output = "document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from train.json
Loaded 24111 records from validation.json
Loaded 24084 records from test.json
Wrote 131921 records to document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood thereon, Your blood again

## You should check that the collection you have matches that of the paper!

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [4]:
import json
import re
import unicodedata
import string

input_file = "test.json"
output_file = "test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [5]:
input_file = "test.json"
qrels_file = "test_qrels.json"
answers_file = "test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to test_qrels.json
Saved 24084 entries to test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval - Good Luck!

# Create the Index of the collection



In [9]:
#path index folder
folder_name = "Index"
index_file_name = "terrier_inverted_index"
index_path = os.path.abspath(os.path.join(folder_name, index_file_name))

#init pyTer
if not pt.java.started():
    pt.java.init()

def get_index():
    #check if the index exsists
    properties_file = os.path.join(index_path, "data.properties")
    # Forcing a rebuild by removing the index directory if it exists to ensure updated schema.
    if os.path.exists(index_path):
        print(f"Existing index found at {index_path}. Removing to ensure rebuild with updated schema.")
        shutil.rmtree(index_path)

    print("Index is not found or is being rebuilt, creating a new Index")
    os.makedirs(index_path, exist_ok=True)

    with open('document_collection.json', 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    # Build Data Frame to include docno, context, raw_ocr, and publication_date
    df = pd.DataFrame(raw_data)
    df = df.rename(columns={"para_id": "docno"})[["docno", "context", "raw_ocr", "publication_date"]]

    df['text'] = df['context'] # Create a 'text' column for the primary indexed field

    # Build the index
    indexer = pt.IterDictIndexer(
        index_path,
        meta={"docno": 24, "context": 1024, "raw_ocr": 1024},
        text_attrs=["text"],           # 'text' (our 'context') is the primary field for indexing content
        meta_reverse=["docno"],        # enable reverse lookup on docno
        pretokenised=False,
        fields=False,
        threads=1,
    )
    index_ref = indexer.index(df.to_dict(orient="records"))
    return pt.IndexFactory.of(index_ref)
index = get_index()

# Print a simple summary
print("Index location:", index_path)
print("Indexed documents:", index.getCollectionStatistics().getNumberOfDocuments())

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Index is not found or is being rebuilt, creating a new Index


/usr/local/lib/python3.12/dist-packages/pyterrier/terrier/index.py:640: UserWarning: Indexing meta key context length requested 1024 but exceeded in first document (actual length 1189). Increase the length in the meta dict for the indexer, e.g., pt.IterDictIndexer(..., meta={'context': 1189}).
  warn(msg)
/usr/local/lib/python3.12/dist-packages/pyterrier/terrier/index.py:640: UserWarning: Indexing meta key raw_ocr length requested 1024 but exceeded in first document (actual length 1256). Increase the length in the meta dict for the indexer, e.g., pt.IterDictIndexer(..., meta={'raw_ocr': 1256}).
  warn(msg)


10:02:03.213 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (Michigan_18370308_24) - further warnings are suppressed
10:02:11.686 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents
Index location: /content/Index/terrier_inverted_index
Indexed documents: 131921


In [ ]:
# Retrieve collection statistics
stats = index.getCollectionStatistics()

print("Terrier Collection Statistics")
print("--------------------------------")
print(f"Indexed documents:        {stats.getNumberOfDocuments()}")
print(f"Unique terms (vocabulary): {stats.getNumberOfUniqueTerms()}")
print(f"Total tokens:             {stats.getNumberOfTokens()}")
print(f"Average document length:  {stats.getAverageDocumentLength():.2f}")

Terrier Collection Statistics
--------------------------------
Indexed documents:        131921
Unique terms (vocabulary): 236646
Total tokens:             15575099
Average document length:  118.06


# SETUP QUERY PIPELINE (Query expansion histword -> query filtering)

## Set up query expansion with histwords

In [ ]:
!wget http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
!unzip -n eng-fiction-all_sgns.zip
!ls -F

--2026-01-22 14:09:52--  http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400120997 (382M) [application/zip]
Saving to: ‘eng-fiction-all_sgns.zip’

eng-fiction-all_sgn 100%[===================>] 381.58M  19.8MB/s    in 27s     

2026-01-22 14:10:18 (14.4 MB/s) - ‘eng-fiction-all_sgns.zip’ saved [400120997/400120997]

Archive:  eng-fiction-all_sgns.zip
   creating: sgns/
  inflating: sgns/1990-vocab.pkl     
  inflating: sgns/1970-w.npy         
  inflating: sgns/1810-w.npy         
  inflating: sgns/1840-vocab.pkl     
  inflating: sgns/1920-vocab.pkl     
  inflating: sgns/1950-w.npy         
  inflating: sgns/1980-w.npy         
  inflating: sgns/1950-vocab.pkl     
  inflating: sgns/1830-w.npy         
  inflating: sgns/1830-vocab.pkl     
  inflating: sgns/1880

In [ ]:
def load_histwords_colab(year):
    path_vocab = f"sgns/{year}-vocab.pkl"
    path_vectors = f"sgns/{year}-w.npy"

    print(f"Loading {year} data...")

    with open(path_vocab, 'rb') as f:
        vocab = pickle.load(f)

    vectors = np.load(path_vectors)

    kv = KeyedVectors(vector_size=vectors.shape[1])
    kv.add_vectors(vocab, vectors)

    print(f"Success! Model for {year} loaded with {len(vocab)} words.")
    return kv

hist_model = load_histwords_colab(1890)

Loading 1890 data...
Success! Model for 1890 loaded with 100000 words.


In [ ]:
hist_models_by_year = {}
for year in range(1800, 1901, 10):
    try:
        hist_models_by_year[year] = load_histwords_colab(year)
    except Exception as e:
        print(f"Could not load model for year {year}: {e}")

print("All requested historical models loaded:")
for year, model in hist_models_by_year.items():
    print(f"  Year {year}: {len(model.key_to_index)} words loaded.")

Loading 1800 data...
Success! Model for 1800 loaded with 100000 words.
Loading 1810 data...
Success! Model for 1810 loaded with 100000 words.
Loading 1820 data...
Success! Model for 1820 loaded with 100000 words.
Loading 1830 data...
Success! Model for 1830 loaded with 100000 words.
Loading 1840 data...
Success! Model for 1840 loaded with 100000 words.
Loading 1850 data...
Success! Model for 1850 loaded with 100000 words.
Loading 1860 data...
Success! Model for 1860 loaded with 100000 words.
Loading 1870 data...
Success! Model for 1870 loaded with 100000 words.
Loading 1880 data...
Success! Model for 1880 loaded with 100000 words.
Loading 1890 data...
Success! Model for 1890 loaded with 100000 words.
Loading 1900 data...
Success! Model for 1900 loaded with 100000 words.
All requested historical models loaded:
  Year 1800: 100000 words loaded.
  Year 1810: 100000 words loaded.
  Year 1820: 100000 words loaded.
  Year 1830: 100000 words loaded.
  Year 1840: 100000 words loaded.
  Year 18

In [ ]:
selected_year = 1900 # Change this year to explore different historical models
word = "gay"

# Get the historical model for the selected year
current_hist_model = hist_models_by_year.get(selected_year)

if current_hist_model:
    if word in current_hist_model:
        print(f"Synonyms in {selected_year} for '{word}':")
        print(current_hist_model.most_similar(word, topn=3))
    else:
        print(f"'{word}' not found in the {selected_year} historical model.")
else:
    print(f"Historical model for year {selected_year} is not loaded.")

Synonyms in 1900 for 'gay':
[('lively', 0.4499954581260681), ('cheery', 0.43901199102401733), ('lark', 0.4136286675930023)]


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


## Query expansion implementation

In [ ]:
import functools
selected_year = 1900 # Chosen year that will be explained later
current_hist_model = hist_models_by_year.get(selected_year)

def hist_expansion_logic(row, top_n_expanded_words, hist_model):
    original_terms = row['query'].split()
    candidate_expansions = []  # To store (word, score) for all similar words

    for term in original_terms:
        word = term.lower()
        if word in hist_model.key_to_index:
            if np.isclose(hist_model.get_vector(word, norm=False).tolist(), 0).all():
                continue
            try:
                similar_words_and_scores = hist_model.most_similar(word, topn=1)
                for sim_word, score in similar_words_and_scores:
                    if sim_word.lower() not in [t.lower() for t in original_terms]:
                        candidate_expansions.append((sim_word.lower(), score))
            except Exception:
                continue

    unique_expansions_dict = {}
    for word, score in sorted(candidate_expansions, key=lambda x: x[1], reverse=True):
        if word not in unique_expansions_dict:
            unique_expansions_dict[word] = score

    final_expanded_words = list(unique_expansions_dict.keys())[:top_n_expanded_words]

    # Combine original terms and the selected expanded words
    full_query_terms = list(original_terms) + final_expanded_words
    return " ".join(full_query_terms)

hist_query_expansion = pt.apply.query(functools.partial(hist_expansion_logic, top_n_expanded_words=2, hist_model=current_hist_model))


## Query reduction implementation (filtering stopwords)

In [ ]:
def query_reduction_by_weight(row):
    original_query = row['query']
    terms = original_query.split()
    reduced_terms = []

    for term in terms:
        if term.lower() not in stop_words:
            reduced_terms.append(term)
    return " ".join(reduced_terms) if reduced_terms else original_query

query_reduction = pt.apply.query(query_reduction_by_weight)

In [ ]:
example_query = "How many gays lots did Thomas Peirce have in the year 1800"

expanded_query = hist_query_expansion.transform(pd.DataFrame([{'qid': 'test_q', 'query': example_query}]))
reduced_query = query_reduction.transform(pd.DataFrame([{'qid': 'test_q', 'query': expanded_query['query'].iloc[0]}]))

# Print the original query for comparison.
print(f"Original request: {example_query}")
print(f"Expanded request: {expanded_query['query'].iloc[0]}")
print(f"Reduced request: {reduced_query['query'].iloc[0]}")

Original request: How many gays lots did Thomas Peirce have in the year 1800
Expanded request: How many gays lots did Thomas Peirce have in the year 1800 month does
Reduced request: many gays lots Thomas Peirce year 1800 month


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


# First stage retrieval pipelines

## Define 3 pipelines (2 baseline then 1 with query expansion)

In [ ]:
# First stage retriver with BM25
bm25_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="BM25")%5
tf_idf_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="TF_IDF")%5

## Test on a subset of the queries

In [ ]:
#testing the engine
query = "Europe"
bm25_results = bm25_1st_stage_retrieval.search(query)
tf_idf_results = tf_idf_1st_stage_retrieval.search(query)
print('-------------------- bm25 results (top 5) --------------------\n',bm25_results.head(5))
print('-------------------- tf_idf results (top 5) --------------------\n',tf_idf_results.head(5))


-------------------- bm25 results (top 5) --------------------
   qid   docid                     docno  rank      score   query
0   1  122024      New_York_19190610_27     0  10.233243  Europe
1   1   61883         Texas_18771215_13     1   9.759719  Europe
2   1    9637      Maryland_18401110_20     2   9.730148  Europe
3   1  110750      Nebraska_19020225_26     3   9.715430  Europe
4   1   36539  Rhode_Island_18511030_25     4   9.642500  Europe
-------------------- tf_idf results (top 5) --------------------
   qid   docid                     docno  rank     score   query
0   1  122024      New_York_19190610_27     0  5.641964  Europe
1   1   61883         Texas_18771215_13     1  5.380893  Europe
2   1    9637      Maryland_18401110_20     2  5.364589  Europe
3   1  110750      Nebraska_19020225_26     3  5.356474  Europe
4   1   36539  Rhode_Island_18511030_25     4  5.316266  Europe


## Comparing each histword model to choose the most relevant one for query expansion

In [ ]:
pipelines = []
pipeline_names = []

# Add baseline pipelines
pipelines.append(bm25_1st_stage_retrieval)
pipeline_names.append("BM25 Baseline")

pipelines.append(tf_idf_1st_stage_retrieval)
pipeline_names.append("TF-IDF Baseline")

# Add custom BM25 pipelines for each historical word embedding model
for year, hist_model in hist_models_by_year.items():
    # Create a new hist_query_expansion for the current year's model
    current_hist_expansion = pt.apply.query(functools.partial(hist_expansion_logic, top_n_expanded_words=2, hist_model=hist_model))

    # Combine with query reduction and BM25
    custom_bm25_pipeline_for_year = current_hist_expansion >> query_reduction >> bm25_1st_stage_retrieval

    pipelines.append(custom_bm25_pipeline_for_year)
    pipeline_names.append(f"BM25 + HistQE {year}")

print(f"Created {len(pipelines)} pipelines for evaluation:")
for name in pipeline_names:
    print(f"- {name}")


Created 13 pipelines for evaluation:
- BM25 Baseline
- TF-IDF Baseline
- BM25 + HistQE 1800
- BM25 + HistQE 1810
- BM25 + HistQE 1820
- BM25 + HistQE 1830
- BM25 + HistQE 1840
- BM25 + HistQE 1850
- BM25 + HistQE 1860
- BM25 + HistQE 1870
- BM25 + HistQE 1880
- BM25 + HistQE 1890
- BM25 + HistQE 1900


In [ ]:
queries_df = pd.DataFrame(queries)
queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})

qrels_df = pd.DataFrame(qrels)
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

num_queries_subset = 100 # Define the number of queries for the subset
queries_subset_df = queries_df.head(num_queries_subset)

qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results = pt.Experiment(
    pipelines,
    queries_subset_df,
    qrels_subset_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=pipeline_names
)

display(experiment_results)

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists =

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25 Baseline,0.558833,0.52,0.128,0.064,0.64,0.64,0.578709,0.578709
1,TF-IDF Baseline,0.562333,0.52,0.126,0.063,0.63,0.63,0.579284,0.579284
2,BM25 + HistQE 1800,0.544000,0.50,0.122,0.061,0.61,0.61,0.560593,0.560593
3,BM25 + HistQE 1810,0.539000,0.51,0.118,0.059,0.59,0.59,0.551665,0.551665
4,BM25 + HistQE 1820,0.523000,0.48,0.120,0.060,0.60,0.60,0.542021,0.542021
5,BM25 + HistQE 1830,0.530667,0.49,0.122,0.061,0.61,0.61,0.550278,0.550278
6,BM25 + HistQE 1840,0.530333,0.50,0.114,0.057,0.57,0.57,0.540415,0.540415
7,BM25 + HistQE 1850,0.537333,0.50,0.118,0.059,0.59,0.59,0.550593,0.550593
8,BM25 + HistQE 1860,0.547833,0.50,0.124,0.062,0.62,0.62,0.566031,0.566031
9,BM25 + HistQE 1870,0.527833,0.49,0.120,0.060,0.60,0.60,0.545717,0.545717



## Run the full pipelines then save their results as csv files

In [ ]:
bm25_custom_1st_stage_retrieval = (
    hist_query_expansion
    >> query_reduction
    >> bm25_1st_stage_retrieval%5
)

In [ ]:
first_stage_pipelines = [bm25_1st_stage_retrieval,tf_idf_1st_stage_retrieval,bm25_custom_1st_stage_retrieval]
output_files=["bm25_candidates","tf-idf_candidates","custom_bm25_candidates"]

queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

for pipeline, output_file in zip(first_stage_pipelines, output_files):
    results = pipeline.transform(queries_df)
    results.to_csv(output_file + ".csv", index=False)

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


## RESULTS OF THE 1ST STAGE RETRIEVAL



In [ ]:
bm25_precomputed_df = pd.read_csv("bm25_candidates.csv")
tf_idf_precomputed_df = pd.read_csv("tf-idf_candidates.csv")
bm25_custom_precomputed_df = pd.read_csv("custom_bm25_candidates.csv")

# Create PyTerrier Transformers from the precomputed DataFrames
bm25_precomputed_pipeline = pt.Transformer.from_df(bm25_precomputed_df)
tf_idf_precomputed_pipeline = pt.Transformer.from_df(tf_idf_precomputed_df)
bm25_custom_precomputed_pipeline = pt.Transformer.from_df(bm25_custom_precomputed_df)

experiment_results = pt.Experiment(
    [bm25_precomputed_pipeline, tf_idf_precomputed_pipeline, bm25_custom_precomputed_pipeline],
    queries_df,
    qrels_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_Precomputed","TF-IDF_Precomputed","BM25_QueryExpansion_histwords_Precomputed"]
)

In [ ]:
display(experiment_results)
experiment_results.to_csv("experiment_results_first_stage.csv", index=False) # DON'T FORGET TO DOWNLOAD IT

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25_Precomputed,0.625393,0.5697,0.14242,0.07121,0.7121,0.7121,0.647150,0.647150
1,TF-IDF_Precomputed,0.625463,0.5700,0.14248,0.07124,0.7124,0.7124,0.647257,0.647257
2,BM25_QueryExpansion_histwords_Precomputed,0.570840,0.5120,0.13292,0.06646,0.6646,0.6646,0.594320,0.594320


# Second stage retrieval pipelines (by using results generated by BM25)

Load result files generated by BM25 first retrieval (both baseline and custom)

In [7]:
input_files=["bm25_candidates.csv","custom_bm25_candidates.csv"]

raw_bm25_results_df = pd.read_csv(input_files[0])
bm25_precomputed_source = raw_bm25_results_df[['qid', 'docid', 'docno', 'score', 'rank']].copy()
raw_custom_bm25_results_df = pd.read_csv(input_files[1])
custom_bm25_precomputed_source = raw_custom_bm25_results_df[['qid', 'docid', 'docno', 'score', 'rank']].copy()

## Define baseline pipeline

This pipeline answers only with top 1 document reranked (bm25 -> monoT5)

In [11]:
# Initialize MonoT5ReRanker
monoT5_reranker = MonoT5ReRanker(batch_size=32)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
context_baseline_rerank = (
    pt.Transformer.from_df(bm25_precomputed_source)
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> monoT5_reranker%1
)

raw_ocr_baseline_rerank = (
    pt.Transformer.from_df(bm25_precomputed_source)
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={'raw_ocr': 'text'}))
    >> monoT5_reranker%1
)

## Run the full pipelines then save their results as csv files

In [ ]:
second_stage_pipelines = [context_baseline_rerank,raw_ocr_baseline_rerank]
output_files=["monot5_reranked_context_candidates","monot5_reranked_raw_ocr_candidates"]

queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

for pipeline, output_file in zip(second_stage_pipelines, output_files):
    results = pipeline.transform(queries_df)
    results.to_csv(output_file + ".csv", index=False)

## RESULTS OF THE 2ST RETRIEVAL BASELINE


In [ ]:
monot5_context_precomputed_df = pd.read_csv("monot5_reranked_context_candidates.csv")
monot5_raw_ocr_precomputed_df = pd.read_csv("monot5_reranked_raw_ocr_candidates.csv")

# Create PyTerrier Transformers from the precomputed DataFrames
monot5_context_precomputed_pipeline = pt.Transformer.from_df(monot5_context_precomputed_df)
monot5_raw_ocr_precomputed_pipeline = pt.Transformer.from_df(monot5_raw_ocr_precomputed_df)

experiment_results = pt.Experiment(
    [monot5_context_precomputed_pipeline, monot5_raw_ocr_precomputed_pipeline],
    queries_df,
    qrels_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["MonoT5_Reranked_Context","MonoT5_Reranked_RawOCR"]
)

In [ ]:
display(experiment_results)
experiment_results.to_csv("experiment_results_baseline_second_stage.csv", index=False) # DON'T FORGET TO DOWNLOAD IT

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,MonoT5_Reranked_Context,0.63315,0.6061,0.13330,0.06665,0.6665,0.6665,0.641734,0.641734
1,MonoT5_Reranked_RawOCR,0.57835,0.5363,0.12626,0.06313,0.6313,0.6313,0.591957,0.591957


## Define custom bm25 reranking pipelines

In [12]:
context_custom_rerank = (
    pt.Transformer.from_df(custom_bm25_precomputed_source)
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> monoT5_reranker%3
)

raw_ocr_custom_rerank = (
    pt.Transformer.from_df(custom_bm25_precomputed_source)
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={'raw_ocr': 'text'}))
    >> monoT5_reranker%3
)

In [14]:
second_stage_pipelines = [context_custom_rerank,raw_ocr_custom_rerank]
output_files=["custom_monot5_reranked_context_candidates","custom_monot5_reranked_raw_ocr_candidates"]

queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

for pipeline, output_file in zip(second_stage_pipelines, output_files):
    results = pipeline.transform(queries_df)
    results.to_csv(output_file + ".csv", index=False)

monoT5:   0%|          | 0/1563 [00:00<?, ?batches/s]

monoT5:   0%|          | 0/1563 [00:00<?, ?batches/s]

In [16]:
custom_monot5_context_precomputed_df = pd.read_csv("custom_monot5_reranked_context_candidates.csv")
custom_monot5_raw_ocr_precomputed_df = pd.read_csv("custom_monot5_reranked_raw_ocr_candidates.csv")

# Create PyTerrier Transformers from the precomputed DataFrames
custom_monot5_context_pipeline = pt.Transformer.from_df(custom_monot5_context_precomputed_df)
custom_monot5_raw_ocr_pipeline = pt.Transformer.from_df(custom_monot5_raw_ocr_precomputed_df)

experiment_results = pt.Experiment(
    [custom_monot5_context_pipeline, custom_monot5_raw_ocr_pipeline],
    queries_df,
    qrels_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["Custom_MonoT5_Reranked_Context","Custom_MonoT5_Reranked_RawOCR"]
)

In [17]:
display(experiment_results)
experiment_results.to_csv("experiment_results_custom_second_stage.csv", index=False) # DON'T FORGET TO DOWNLOAD IT

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,Custom_MonoT5_Reranked_Context,0.598117,0.5748,0.12534,0.06267,0.6267,0.6267,0.605477,0.605477
1,Custom_MonoT5_Reranked_RawOCR,0.552717,0.5159,0.11980,0.05990,0.5990,0.5990,0.564612,0.564612


### LLM MODEL DEFINITION FOR QUESTION ANSWERING (GEMMA3 270M)

In [6]:
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
def gemma_qa_logic(df):
    prompts = [
        f"""Given the following Context:
{str(row['text'])[:1948]}

Question: {row['query']}

Extract the most concise, direct, factual answer from the Context.
If the answer is not present in the Context, respond with 'Answer not found'.
Your answer must be ONLY the extracted factual information, or 'Answer not found', with no additional text or preamble.
Answer:"""
        for _, row in df.iterrows()
    ]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True,max_length=1024).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=False,
            use_cache=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id
        )
    input_len = inputs.input_ids.shape[1]
    decoded_outputs = tokenizer.batch_decode(outputs[:, input_len:], skip_special_tokens=True)

    cleaned_answers = []
    for ans_text in decoded_outputs:
        # Remove any leading/trailing whitespace
        ans_text = ans_text.strip()
        # Look for the 'Answer:' prefix that the LLM might generate based on the prompt
        if ans_text.lower().startswith('answer:'):
            ans_text = ans_text[len('answer:'):].strip()
        # Take only the first line if there are multiple lines generated
        ans_text = ans_text.split('\n')[0].strip()
        cleaned_answers.append(ans_text)

    df["answer"] = cleaned_answers
    return df

==((====))==  Unsloth 2026.1.4: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/388M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

### DEFINE CUSTOM PIPELINE

This pipeline use LLM model to answer based on reranked documents (top1 then top1-3 if possible)

In [18]:
monot5_context_reranked_df_full = pd.read_csv("monot5_reranked_context_candidates.csv")
monot5_raw_ocr_reranked_df_full = pd.read_csv("monot5_reranked_raw_ocr_candidates.csv")

# --- Define 4 pipelines ---

# 1. QA with Context (Top 1 document)
monot5_context_top1_df = monot5_context_reranked_df_full.loc[monot5_context_reranked_df_full.groupby('qid')['score'].idxmax()].reset_index(drop=True)
qa_context_top1_from_monot5 = (
    pt.Transformer.from_df(monot5_context_top1_df) >> pt.apply.generic(gemma_qa_logic, batch_size=32)
)

# 2. QA with Raw OCR (Top 1 document)
monot5_raw_ocr_top1_df = monot5_raw_ocr_reranked_df_full.loc[monot5_raw_ocr_reranked_df_full.groupby('qid')['score'].idxmax()].reset_index(drop=True)
qa_raw_ocr_top1_from_monot5 = (
    pt.Transformer.from_df(monot5_raw_ocr_top1_df) >> pt.apply.generic(gemma_qa_logic, batch_size=32)
)

# 3. QA with Context (Top N documents merged)
def merge_top_texts(df):
    merged_data = []

    for qid, group in df.groupby("qid"):
        query = group["query"].iloc[0]

        full_context = ""
        for i, row in enumerate(group.itertuples()):
            full_context += f"\n--- Document {i+1} ---\n{row.text}"

        merged_data.append({
            "qid": qid,
            "query": query,
            "text": full_context.strip(),
            "docno": f"merged_{qid}"
        })

    return pd.DataFrame(merged_data)
# This pipeline directly uses the full reranked DataFrame and then merges texts
qa_merged_context_from_monot5 = (
    pt.Transformer.from_df(monot5_context_reranked_df_full)
    >> pt.apply.generic(merge_top_texts) # merge_top_texts expects a 'text' column
    >> pt.apply.generic(gemma_qa_logic, batch_size=64)
)

# 4. QA with Raw OCR (Top N documents merged)
qa_merged_raw_ocr_from_monot5 = (
    pt.Transformer.from_df(monot5_raw_ocr_reranked_df_full)
    >> pt.apply.generic(merge_top_texts) # merge_top_texts expects a 'text' column
    >> pt.apply.generic(gemma_qa_logic, batch_size=64)
)

print("Four LLM QA pipelines created: Context Top1, Raw OCR Top1, Merged Context (Top N), Merged Raw OCR (Top N).")

Four LLM QA pipelines created: Context Top1, Raw OCR Top1, Merged Context (Top N), Merged Raw OCR (Top N).


### TESTING QA model on a subset of the queries

In [22]:
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

num_queries_subset = 100
queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

# Run the experiment with the QA re-ranking pipeline
experiment_results = pt.Experiment(
    [qa_context_top1_from_monot5,qa_raw_ocr_top1_from_monot5],
    queries_subset_df,  # Use the subset of queries
    qrels_subset_df,    # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["Custom RAG with top 1 doc context","Custom RAG with top 1 doc raw ocr"],
    # precompute_prefix=True,
    validate='ignore'
)

display(experiment_results)

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,Custom RAG with top 1 doc context,0.55,0.55,0.11,0.055,0.55,0.55,0.55,0.55
1,Custom RAG with top 1 doc raw ocr,0.45,0.45,0.09,0.045,0.45,0.45,0.45,0.45


In [ ]:
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

llm_qa_pipelines = [qa_context_top1_from_monot5, qa_raw_ocr_top1_from_monot5]
llm_output_files = ["llm_answers_context_top1", "llm_answers_raw_ocr_top1"]

for pipeline, output_file in zip(llm_qa_pipelines, llm_output_files):
    results = pipeline.transform(queries_df)
    results.to_csv(output_file + ".csv", index=False)
    print(f"Saved '{output_file}.csv'")

print("Transformation complete. Check the generated CSV files for LLM answers.")

### Using 3 documents instead of 1 only document for Question Answering by LLM (BETA TESTING)

To doing so, we have to merge the texts of each documents in one prompt in order to allow LLM model to consider all of them.

In [ ]:
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

num_queries_subset = 10 # Using the same subset size for consistency
queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

print(f"\nTransforming queries with {num_queries_subset} queries using the new LLM pipelines and saving to CSV...")

# Transform with the context-based pipeline and save
results_context_llm = qa_merged_context_pipeline_from_monot5_precomputed.transform(queries_subset_df)
results_context_llm.to_csv("llm_answers_context_merged.csv", index=False)
print("Saved 'llm_answers_context_merged.csv'")
display(results_context_llm.head(5))
# Transform with the raw_ocr-based pipeline and save
results_raw_ocr_llm = qa_merged_raw_ocr_pipeline_from_monot5_precomputed.transform(queries_subset_df)
results_raw_ocr_llm.to_csv("llm_answers_raw_ocr_merged.csv", index=False)
print("Saved 'llm_answers_raw_ocr_merged.csv'")
display(results_raw_ocr_llm.head(5))

print("Transformation complete. Check the generated CSV files for LLM answers.")


Transforming queries with 10 queries using the new LLM pipelines and saving to CSV...
Saved 'llm_answers_context_merged.csv'


,qid,query,text,docno,answer
0,test_1,How many lots did Thomas Peirce have,"--- Document 1 ---\nIn Gloucester county, (N.J...",merged_test_1,--- Final Answer Not Found}
1,test_10,Who gave Hamilton the substance of what he had...,--- Document 1 ---\nGeneral Hamilton would dec...,merged_test_10,What material does this suggest?
2,test_100,Who informs his FRIENDS and the PUBLIC that he...,"--- Document 1 ---\nDr. Ree's Cyclopædia, No. ...",merged_test_100,What was it?
3,test_1000,Who was the Secretary of the Treasury of the U...,--- Document 1 ---\nOpposite the Jefferson Hot...,merged_test_1000,Answer not available.
4,test_10000,Who made a speech in front of the Brooks House,--- Document 1 ---\nMr. Brooks divided his hou...,merged_test_10000,What was the main point of the Speech?\n**Answ...


Saved 'llm_answers_raw_ocr_merged.csv'


,qid,query,text,docno,answer
0,test_1,How many lots did Thomas Peirce have,"--- Document 1 ---\nIn Gloucelier county, (N. ...",merged_test_1,90
1,test_10,Who gave Hamilton the substance of what he had...,--- Document 1 ---\nGeneral Hamilton\nwould de...,merged_test_10,What was the substance o**t what he has sugges...
2,test_100,Who informs his FRIENDS and the PUBLIC that he...,"--- Document 1 ---\n* _\n, Dr. Rze's .Cyglopcd...",merged_test_100,The name of the firm where it was headquartered.
3,test_1000,Who was the Secretary of the Treasury of the U...,--- Document 1 ---\nThat the President and Vic...,merged_test_1000,Answer not available because there is no speci...
4,test_10000,Who made a speech in front of the Brooks House,--- Document 1 ---\nMr. Brooks divided his hou...,merged_test_10000,What were Mr. Brooks’s words?


Transformation complete. Check the generated CSV files for LLM answers.


To evaluate the generated answers, we'll implement functions to calculate Exact Match (EM) and F1 score. We'll then load the generated answers and the ground truth answers to compute these metrics for both pipelines.

In [ ]:
import re
import string
import json
import pandas as pd
from collections import Counter

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    return white_space_fix(remove_articles(remove_punc(s.lower())))

def compute_exact_match(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_f1(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_common = sum(common.values())
    if num_common == 0:
        return 0
    precision = num_common / len(prediction_tokens)
    recall = num_common / len(ground_truth_tokens)
    return (2 * precision * recall) / (precision + recall)

# Load ground truth answers
with open("test_query_answers.json", "r", encoding="utf-8") as f:
    ground_truth_data = json.load(f)

ground_truth_df = pd.DataFrame(ground_truth_data)
ground_truth_df = ground_truth_df.rename(columns={"query_id": "qid"})

# Filter ground truth to match the subset of queries used for LLM generation
# We need to ensure we use the same subset of query IDs as used for 'queries_subset_df'
# Re-create queries_subset_df here to ensure consistency if this cell is run independently
queries_df = pd.DataFrame(queries).rename(columns={"query_id": "qid", "question": "query"})
num_queries_subset = 100
queries_subset_qids = queries_df.head(num_queries_subset)['qid'].tolist()

ground_truth_filtered_df = ground_truth_df[ground_truth_df['qid'].isin(queries_subset_qids)]

# Map ground truth to a dictionary for easy lookup by qid
ground_truth_answers = {
    row['qid']: row['org_answer'] for idx, row in ground_truth_filtered_df.iterrows()
}

# Load generated answers
llm_answers_context_df = pd.read_csv("llm_answers_context_merged.csv")
llm_answers_raw_ocr_df = pd.read_csv("llm_answers_raw_ocr_merged.csv")

def evaluate_pipeline_answers(llm_answers_df, ground_truth_dict, pipeline_name):
    em_scores = []
    f1_scores = []
    for idx, row in llm_answers_df.iterrows():
        qid = row['qid']
        prediction = str(row['answer'])
        if qid in ground_truth_dict:
            ground_truth = str(ground_truth_dict[qid])
            em_scores.append(compute_exact_match(prediction, ground_truth))
            f1_scores.append(compute_f1(prediction, ground_truth))
        else:
            # If a qid is in LLM output but not in filtered ground truth, skip or handle as needed
            # For this context, it implies a mismatch in the subsets, which should ideally not happen.
            pass # We expect qids to align due to subsetting

    if len(em_scores) > 0:
        avg_em = sum(em_scores) / len(em_scores)
        avg_f1 = sum(f1_scores) / len(f1_scores)
    else:
        avg_em = 0.0
        avg_f1 = 0.0

    print(f"\n--- {pipeline_name} ---")
    print(f"Average Exact Match: {avg_em:.4f}")
    print(f"Average F1 Score: {avg_f1:.4f}")
    return {"pipeline": pipeline_name, "EM": avg_em, "F1": avg_f1}

results_list = []
results_list.append(evaluate_pipeline_answers(llm_answers_context_df, ground_truth_answers, "LLM with Context (Merged)"))
results_list.append(evaluate_pipeline_answers(llm_answers_raw_ocr_df, ground_truth_answers, "LLM with Raw OCR (Merged)"))

results_df = pd.DataFrame(results_list)
display(results_df)



--- LLM with Context (Merged) ---
Average Exact Match: 0.0000
Average F1 Score: 0.0000

--- LLM with Raw OCR (Merged) ---
Average Exact Match: 0.0000
Average F1 Score: 0.0000


,pipeline,EM,F1
0,LLM with Context (Merged),0.0,0.0
1,LLM with Raw OCR (Merged),0.0,0.0


After executing the code above, you'll see a table comparing the 'LLM with Context (Merged)' and 'LLM with Raw OCR (Merged)' pipelines based on their Average Exact Match (EM) and Average F1 Scores.

**How to compare:**

*   **Higher EM is better:** A higher Exact Match score indicates that the LLM is generating answers that are verbatim identical to the ground truth more often. This is a very strict measure of accuracy.
*   **Higher F1 is better:** A higher F1 score indicates a better balance between precision and recall in the generated answer's token overlap with the ground truth. This is generally a more forgiving metric and often preferred for QA tasks where slight variations in phrasing are acceptable.

By comparing these two metrics across your 'Context' and 'Raw OCR' pipelines, you can determine which approach leads to more accurate answers from your LLM. For instance, if 'LLM with Context' has a higher F1, it suggests that using the cleaned 'context' field provides a better basis for the LLM to generate semantically similar, correct answers, even if they aren't always exact matches.

Let's inspect the output of the `merge_top_texts` function. We'll take a subset of queries, apply the MonoT5 re-ranking, and then apply `merge_top_texts` to see how the documents are combined.

In [ ]:
# Take a small subset of queries for demonstration
queries_subset_df_for_merge_test = queries_df.head(1)

# Create a temporary pipeline to show the output after merge_top_texts
temp_merge_pipeline = (
    pt.Transformer.from_df(monot5_context_reranked_df)  # Start with all reranked documents
    >> pt.apply.generic(merge_top_texts)
)

# Apply the pipeline to the subset of queries
merged_output_df = temp_merge_pipeline.transform(queries_subset_df_for_merge_test)

print("DataFrame after merge_top_texts (first query):")
# Display the relevant columns, especially the 'text' column
display(merged_output_df[['qid', 'query', 'docno', 'text']].head())

print("\nContent of the merged 'text' for the first query:")
print(merged_output_df['text'].iloc[0])

print("\nAlso, let's verify how many documents were fed to merge_top_texts for the first query.")
# The original monot5_context_reranked_df should contain 3 documents per query at this stage
initial_docs_for_first_query = monot5_context_reranked_df[monot5_context_reranked_df['qid'] == queries_subset_df_for_merge_test['qid'].iloc[0]]
print(f"Number of documents initially retrieved for the first query (before merging): {len(initial_docs_for_first_query)}")

DataFrame after merge_top_texts (first query):


,qid,query,docno,text
0,test_1,How many lots did Thomas Peirce have,merged_test_1,"--- Document 1 ---\nIn Gloucester county, (N.J..."



Content of the merged 'text' for the first query:
--- Document 1 ---
In Gloucester county, (N.J.) on the 8th ult. THOMAS SEEDE, a native of the county, aged 104 years, 1 month & 20 days. It is worthy of remark that there have died in this county within the space of two years, three citizens, whose ages together amount to upwards of 317 years.  In this town, Mr. GEORGE Hart; A child of Mr. Gold, and a child of Mr. Thomas Peirce. NOTICE—MAPS OF THE UNITED STATES. NEW, correct and beautifully varnished MAPS of the United States, various prices just received and for sale at the Book store of C. PEIRCE, Daniel street, Portsmouth, where may also be had Account Books, of almost all sizes, prices and qualities. Also A fresh supply of BOOKS & STATIONERY; Penknives; Razors; Scissors; Inkstands; Ink Powder; Wafers; Black- Sand; Marking Ink for Linen; Wash- balls; best Windsor Soap; Ivory and bone paper Folders and Cutters; silver Pencil Cases and Pencils; Morocco-Pocket and Memorandum Books; Lad

### Using ASPIRE

PyTerrier : https://pyterrier.readthedocs.io/en/latest/experiments.html

ASPIRE https://github.com/GiorgosPeikos/ASPIRE

ONLINE: https://aspire-ir-eval.streamlit.app/



Remember the res files are actually the rankings created by our retrieval model!

We downloading it to use another tool for data analysis. The tool is called ASPIRE.

ASPIRE requires a different input format!

In short: this code generates results, queries, qrels files which can be uploaded to ASPIRE

In [7]:
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom

# --- Define queries_df and qrels_df ---
# These DataFrames are essential for generating queries.xml and qrels.csv.
# They are created from the global 'queries' and 'qrels' lists,
# which are loaded from your test.json file at the beginning of the notebook.
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

# Rename columns to match PyTerrier and ASPIRE's expected formats
queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

# ----- Function to Save Retrieval Results (for ASPIRE) -----
def save_res(df, run_name, out_path, sep="\t"):
    res = df.copy()  # Create a copy to avoid modifying the original DataFrame
    # Normalize 'docid' to 'docno' if necessary
    if "docid" in res.columns and "docno" not in res.columns:
        res = res.rename(columns={"docid": "docno"})
    # Remove query text columns as they are not needed in the result file
    res = res.drop(columns=["query", "query_0"], errors="ignore")
    res["iteration"] = "Q0"  # Set fixed iteration value as required by ASPIRE
    res["experiment_id"] = run_name  # Label for the experimental run
    res = res.rename(columns={"qid": "query_id", "docno": "doc_id"})  # Rename for ASPIRE compatibility
    res["score"] = pd.to_numeric(res["score"]) # Ensure score is numeric
    # Sort results by query_id and score to determine rank
    res = res.sort_values(by=["query_id", "score"], ascending=[True, False])
    # Assign ranks based on sorted scores within each query
    res["rank"] = res.groupby("query_id")["score"].rank(ascending=False, method="first").astype(int)
    # Select and order columns as required by ASPIRE
    res = res[["query_id", "iteration", "doc_id", "rank", "score", "experiment_id"]]
    res.to_csv(out_path, sep=sep, index=False, header=False)  # Save to CSV without header

# --- Load Precomputed Candidates from all first-stage pipelines ---
input_result_files = [
    ("bm25_candidates.csv", "BM25_Baseline", "results_bm25_baseline.csv"),
    ("tf-idf_candidates.csv", "TFIDF_Baseline", "results_tf_idf_baseline.csv"),
    ("custom_bm25_candidates.csv", "BM25_HistQE", "results_bm25_hist_qe.csv")
]

for file_path, run_name, output_filename in input_result_files:
    try:
        # Load the raw results DataFrame
        raw_results_df = pd.read_csv(file_path)

        # Prepare the DataFrame for processing by save_res
        processed_df = raw_results_df[['qid', 'docid', 'docno', 'score', 'rank']].copy()
        save_res(processed_df, run_name, output_filename)
        print(f"Successfully loaded and reformatted {file_path}. Generated {output_filename}.")
    except Exception as e:
        print(f"Could not load or process {file_path}: {e}")

# ----- Generate Queries.xml (for ASPIRE) -----
# Create the root element for the XML file with a task attribute
root = ET.Element("topics", attrib={"task": "Chronicling America QA"})

# Iterate through each query in 'queries_df' and add it as a '<topic>' element
for _, row in queries_df.iterrows():
    topic = ET.SubElement(root, "topic", attrib={"number": str(row["qid"])}) # Add topic node with qid
    topic.text = str(row["query"])  # Add query text as content of the topic node

# Pretty print the XML for better readability
rough_xml = ET.tostring(root, encoding="utf-8")
pretty_xml = minidom.parseString(rough_xml).toprettyxml(indent="  ", encoding="utf-8")

# Write the formatted XML to a file
with open("queries.xml", "wb") as f:
    f.write(pretty_xml)

print("Generated queries.xml file.")

# ----- Generate Qrels.csv (for ASPIRE) -----
qrels_out = qrels_df.copy() # Create a copy of the qrels DataFrame
# Remove 'iteration' if it exists, as we'll set it specifically for ASPIRE
qrels_out = qrels_out.drop(columns=["iteration"], errors="ignore")
# Rename 'docid' to 'docno' if it exists for consistency
if "docid" in qrels_out.columns and "docno" not in qrels_out.columns:
    qrels_out = qrels_out.rename(columns={"docid": "docno"})
# Rename columns to match ASPIRE's expected format
qrels_out = qrels_out.rename(columns={"qid": "query_id", "docno": "doc_id"})
qrels_out["iteration"] = "Q0" # Set the iteration column to "Q0" as required
# Select and order columns as required by ASPIRE (no header)
qrels_out = qrels_out[["query_id", "iteration", "doc_id", "relevance"]]
qrels_out.to_csv("qrels.csv", sep=" ", index=False, header=False)

print("Generated qrels.csv file.")

print("All files generated for ASPIRE compatibility.")

Successfully loaded and reformatted bm25_candidates.csv. Generated results_bm25_baseline.csv.
Successfully loaded and reformatted tf-idf_candidates.csv. Generated results_tf_idf_baseline.csv.
Successfully loaded and reformatted custom_bm25_candidates.csv. Generated results_bm25_hist_qe.csv.
Generated queries.xml file.
Generated qrels.csv file.
All files generated for ASPIRE compatibility.


In [ ]:
# To use ASPIRE, we need to download the res files, the qrels and queries in the
# required dataformat and use the interface or use it locally.

# Res file Expected format: query_id, iteration (i.e. Q0), doc_id, rank, score, experiment_id
# Queries Expected format: query_id, query
# Qrels Expected format: query_id, iteration (i.e. Q0), doc_id, relevance (without header row)

# DEMO TIME!